# Modèles de langage *ngram* avec NLTK

Je vous invite à lire le notebook de Yoav Goldberg [ici](http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139) ainsi que l'article de blog « The Unreasonable Effectiveness of Recurrent Neural Networks » [ici](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) auquel il fait référence.

## Entraînement

Pour l'entraînement (ou apprentissage) nous allons utiliser des données issues de 52 articles du journal Le Monde publiés en 2016 dans la rubrique 'Pixels'. Le fichier ``data.txt.tag`` contient le contenu textuel de ces articles étiqueté avec TreeTagger.

### Liste de mots

Génération d'une liste de mots (tokens) à partir du fichier ``data.txt.tag``

In [62]:
import nltk
import random

def word_list(file):
    """
    Extrait la liste des tokens d'un fichier TreeTagger (passage en minuscule)
    Args:
        file (str): fichier TreeTagger
    Returns:
        list: la liste des tokens
    """
    words = list()
    with open(file) as f:
        for line in f:
            line = line.rstrip()
            if len(line.split('\t')) == 3:
                token, tag, lemma = line.split('\t')
                words.append(token.lower())
    return words

In [2]:
words = word_list('data.txt.tag')
print(len(words))

95115


### Entraînement d'un modèle bigram

Nous allons nous appuyer sur la fonction ``nltk.bigrams`` pour générer la liste des bigrammes possibles de notre liste de mots.  
Il faut d'abord calculer la fréquence de ces bigrammes avant de calculer leur probabilité qui nous donnera notre modèle de langage.

In [ ]:
cfreq_2gram = nltk.ConditionalFreqDist(nltk.bigrams(words))
cfreq_2gram['il']


In [ ]:
cprob_2gram_mle = nltk.ConditionalProbDist(cfreq_2gram, nltk.MLEProbDist)
cprob_2gram_mle['il'].prob('machin')
cprob_2gram_mle['il'].samples()


In [ ]:
cprob_2gram_laplace = nltk.ConditionalProbDist(cfreq_2gram, nltk.LaplaceProbDist)
cprob_2gram_laplace['il'].samples()

### Entraînement d'un modèle trigram

La fonction ``nltk.trigrams`` renvoie une liste de tuples à trois éléments. Cette structure ne peut pas être utilisée directement par le constructeur de la classe nltk.ConditionalFreqDist.  
Pour le modèle trigram on doit simplement ajouter une petite étape pour produire la structure de données attendue.

In [3]:
trigrams = nltk.trigrams(words)
cfreq_3gram = nltk.ConditionalFreqDist(((w1, w2), w3) for w1, w2, w3 in trigrams)

In [ ]:
cfreq_3gram[('de', 'la')]

In [33]:
cprob_3gram_laplace = nltk.ConditionalProbDist(cfreq_3gram, nltk.LaplaceProbDist)
cprob_3gram_laplace[('sur', 'moi')].max()

'pour'

## Génération de texte avec modèle ngram

L'idée ici est de se servir du modèle ngram pour générer un texte à partir d'une amorce.  
Sans aucune structure syntaxique donc, rien d'autre qu'un modèle trigram appris 95 000 mots. Est-ce que cela va fonctionner ?

In [46]:
def generate_text(history, model, size=100):
    history_words = history.split(' ')
    for i in range(size):
        next_word = model[(history_words[-2], history_words[-1])].max()
        history_words.append(next_word)
    return " ".join(history_words)

In [51]:
generate_text("C'était sans compter sur la présence de", cprob_3gram_laplace, 40)

"C'était sans compter sur la présence de députés et trois ministres – intérieur , bernard cazeneuve a aussi rappelé l’ omniprésence des attaques purement criminelles . dans un communiqué . utilisé par des pirates , qui a été piraté par l' armée électronique syrienne , elle a"

Ça fonctionne à peu près. Enfin ça fait illusion plutôt, on peut facilement mettre à défaut le modèle

In [52]:
generate_text("c'est de la", cprob_3gram_laplace, 40)

"c'est de la nsa , et de la nsa , et de la nsa , et de la nsa , et de la nsa , et de la nsa , et de la nsa , et de la nsa , et de la"

Une des parades serait de ne pas sélectionner le mot le plus probable mais d'en piocher un parmi les n plus probables

In [86]:
def get_rand_nbest(hist, model, n):
    """
    Returns a random word among the n bests according to the model
    Args:
        hist (tuple): the words preceding (tuple of 2 words for a 3gram model)
        model (ConditionalFreqDist): ngram model
        n (int): n best words
    Returns:
        str
    """
    bests = sorted(model[hist].samples(), key=lambda sample: model[hist].prob(sample), reverse=True)
    if len(bests) > n:
        n_best = bests[:n]
        word = n_best[random.randint(0, n-1)]
    else:
        n_best = bests
        word = n_best[random.randint(0, (len(n_best)-1))]
    return word

In [85]:
get_rand_nbest(('de', 'la'), cprob_3gram_laplace, 15)

'loi'

In [87]:
def generate_text(history, model, size=100):
    history_words = history.split(' ')
    for i in range(size):
        next_word = get_rand_nbest((history_words[-2], history_words[-1]), model, 15)
        history_words.append(next_word)
    return " ".join(history_words)

In [90]:
generate_text("c'est de la", cprob_3gram_laplace, 40)

"c'est de la société civile » . la motion , qui a travaillé à ajouter la reconnaissance de caractère à un « s » après un article lui avait valu de vivre des expériences qui semblent n' avoir jamais été publiés ailleurs ,"

In [91]:
generate_text("C'était sans compter sur la présence de", cprob_3gram_laplace, 40)

"C'était sans compter sur la présence de députés et de ce dernier lui est demandé pourquoi il a bien fallu se construire . fuyant l’ empire ottoman et le drapeau syriens , accompagnés d' une procédure disciplinaire de l' assemblée nationale à partir du destin de ces"

C'est mieux ce n'est pas encore tout à fait satisfaisant. Quelles pistes pourrait-on tester pour améliorer notre génération de texte ?